In [35]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio.set_index('Ticker', inplace=True)

portfolio = portfolio[['CVaR','CVaR $']]

portfolio['CVaR'] = portfolio['CVaR'].astype(float)
portfolio['CVaR $'] = portfolio['CVaR $'].astype(float)

portfolio = portfolio[['CVaR','CVaR $']]


portfolio.reset_index(inplace=True)
portfolio.rename(columns={'index': 'Ticker'}, inplace=True)

portfolio

,Ticker,CVaR,CVaR $
0,CEG,0.077,1488.81
1,AAPL,0.071,1378.35
2,RTX,0.068,1325.82
3,PANW,0.048,928.87
4,NOW,0.038,736.67
5,AMZN,0.050,978.19
6,CVX,0.062,1208.52
7,JPM,0.065,1258.19
8,MSFT,0.086,1667.47
9,LOW,0.056,1079.22


In [37]:
portfolio_value = portfolio['CVaR $'].sum()

end = pd.Timestamp(datetime.date.today())
#start = end - relativedelta(years=1)
start = '2023-11-16'

symbols = portfolio['Ticker'].tolist()

#stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data = openbb.stocks.ca.hist(symbols,start, end)

#stock_data.dropna(how="any", axis=1, inplace=True)

stock_data.reset_index(inplace=True)
#stock_data.rename(columns={'index': 'Ticker'}, inplace=True)

stock_data



,Date,CEG,AAPL,RTX,PANW,NOW,AMZN,CVX,JPM,MSFT,LOW,FTNT,ULTA,CMI,NVDA,GEHC
0,2023-11-16,121.669998,189.710007,80.230003,242.300003,652.280029,142.830002,141.770004,150.521835,376.170013,201.267944,50.419998,405.549988,221.742844,494.756500,71.891701
1,2023-11-17,121.690002,189.690002,79.680000,247.589996,654.359985,145.179993,144.460007,151.883438,369.850006,202.680847,51.259998,409.269989,223.807236,492.936676,72.971275
2,2023-11-20,122.279999,191.449997,79.169998,260.570007,666.919983,146.130005,144.380005,152.350555,377.440002,203.417145,51.910000,405.609985,222.477295,504.045685,73.301147
3,2023-11-21,122.510002,190.639999,78.150002,264.029999,668.400024,143.899994,144.039993,152.032516,373.070007,197.069061,52.580002,404.920013,222.119995,499.396088,73.351128
4,2023-11-22,123.250000,191.309998,79.820000,263.140015,669.659973,146.710007,144.330002,152.390305,377.850006,197.486954,52.970001,409.489990,224.509995,487.117188,73.571037
5,2023-11-24,123.860001,189.970001,79.980003,266.079987,673.900024,146.740005,144.899994,152.599014,377.429993,198.074020,53.430000,414.720001,225.490005,477.718018,73.681000
6,2023-11-27,125.099998,189.789993,79.470001,269.089996,670.549988,147.729996,144.360001,152.251175,378.609985,198.820267,53.740002,414.750000,224.410004,482.377594,71.112007
7,2023-11-28,124.449997,190.399994,80.190002,277.829987,665.239990,147.029999,145.509995,152.599014,382.700012,199.049103,52.790001,417.959991,223.750000,478.167938,68.103195
8,2023-11-29,120.580002,189.369995,80.949997,288.910004,678.929993,146.320007,143.910004,153.374252,378.850006,198.919754,53.599998,420.010010,223.949997,481.357666,67.683357
9,2023-11-30,121.040001,189.949997,81.480003,295.089996,685.739990,146.089996,143.600006,155.123459,378.910004,197.835220,52.560001,425.989990,224.160004,467.658905,68.433060


In [38]:
results = pd.DataFrame(columns=['Date', 'Cumulative_Portfolio_Value'])

# Assuming that your price_data DataFrame has a 'Date' column, include it in the results DataFrame.
results['Date'] = stock_data['Date']

# Initialize the first row with starting values
results.at[0, 'Cumulative_Portfolio_Value'] = portfolio_value
for ticker in portfolio['Ticker']:
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    results.at[0, f'{ticker}_Contribution'] = portfolio_value * allocation

# Iterate through stock tickers and calculate portfolio value and contributions
for ticker in portfolio['Ticker']:
    returns_column = f'{ticker}_Return'
    allocation = portfolio.loc[portfolio['Ticker'] == ticker, 'CVaR'].values[0]
    for i in range(1, len(results)):
        results.at[i, f'{ticker}_Return'] = stock_data.at[i, ticker] / stock_data.at[i - 1, ticker] - 1
        results.at[i, f'{ticker}_Contribution'] = results.at[i - 1, f'{ticker}_Contribution'] * (1 + results.at[i, f'{ticker}_Return'])

# Calculate daily portfolio value and cumulative portfolio value
for i in range(1, len(results)):
    cumulative_portfolio_value = portfolio_value
    for ticker in portfolio['Ticker']:
        contribution_column = f'{ticker}_Contribution'
        cumulative_portfolio_value += results.at[i, contribution_column]
    results.at[i, 'Cumulative_Portfolio_Value'] = cumulative_portfolio_value


results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,AMZN_Return,CVX_Return,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return
0,2023-11-16,19415.01,1494.955770,1378.465710,1320.220680,931.920480,737.770380,970.750500,1203.730620,1261.975650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-17,38959.044496,1495.201562,1378.320355,1311.170133,952.266551,740.122943,986.722318,1226.570700,1273.391333,...,0.016453,0.018974,0.009046,-0.016801,0.007020,0.016660,0.009173,0.009310,-0.003678,0.015017
2,2023-11-20,39109.491955,1502.450830,1391.108780,1302.777819,1002.189531,754.329103,993.179118,1225.891427,1277.307646,...,0.006544,-0.000554,0.003075,0.020522,0.003633,0.012680,-0.008943,-0.005942,0.022536,0.004521
3,2023-11-21,39022.7353,1505.276875,1385.223198,1285.993317,1015.497153,756.003124,978.022749,1223.004480,1274.641207,...,-0.015260,-0.002355,-0.002088,-0.011578,-0.031207,0.012907,-0.001701,-0.001606,-0.009225,0.000682
4,2023-11-22,39120.771619,1514.369207,1390.091520,1313.473886,1012.074146,757.428207,997.121127,1225.466864,1277.640903,...,0.019528,0.002013,0.002353,0.012813,0.002121,0.007417,0.011286,0.010760,-0.024587,0.002998
5,2023-11-24,39157.260898,1521.864267,1380.354875,1316.106818,1023.381699,762.223976,997.325015,1230.306512,1279.390726,...,0.000204,0.003949,0.001370,-0.001112,0.002973,0.008684,0.012772,0.004365,-0.019296,0.001495
6,2023-11-27,39122.081405,1537.100085,1379.046906,1307.714504,1034.958627,758.434870,1004.053528,1225.721576,1276.474440,...,0.006747,-0.003727,-0.002279,0.003126,0.003768,0.005802,0.000072,-0.004790,0.009754,-0.034866
7,2023-11-28,39087.678008,1529.113535,1383.479276,1319.562447,1068.573880,752.428924,999.295967,1235.485862,1279.390726,...,-0.004738,0.007966,0.002285,0.010803,0.001151,-0.017678,0.007740,-0.002941,-0.008727,-0.042311
8,2023-11-29,39094.813592,1481.563016,1375.995127,1332.068497,1111.189211,767.913191,994.470479,1221.900774,1285.890325,...,-0.004829,-0.010996,0.005080,-0.010060,-0.000650,0.015344,0.004905,0.000894,0.006671,-0.006165
9,2023-11-30,39141.857289,1487.215012,1380.209521,1340.789990,1134.958347,775.615733,992.907199,1219.268669,1300.555680,...,-0.001572,-0.002154,0.011405,0.000158,-0.005452,-0.019403,0.014238,0.000938,-0.028459,0.011077


In [39]:
# Step 1: Select columns containing the word 'Contribution'
contribution_columns = [col for col in results.columns if 'Contribution' in col]

# Step 2: Create a new column that sums each row of 'Contribution' columns
results['Total_Contribution'] = results[contribution_columns].sum(axis=1)

results

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,CVX_Return,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return,Total_Contribution
0,2023-11-16,19415.01,1494.955770,1378.465710,1320.220680,931.920480,737.770380,970.750500,1203.730620,1261.975650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19434.425010
1,2023-11-17,38959.044496,1495.201562,1378.320355,1311.170133,952.266551,740.122943,986.722318,1226.570700,1273.391333,...,0.018974,0.009046,-0.016801,0.007020,0.016660,0.009173,0.009310,-0.003678,0.015017,19544.034496
2,2023-11-20,39109.491955,1502.450830,1391.108780,1302.777819,1002.189531,754.329103,993.179118,1225.891427,1277.307646,...,-0.000554,0.003075,0.020522,0.003633,0.012680,-0.008943,-0.005942,0.022536,0.004521,19694.481955
3,2023-11-21,39022.7353,1505.276875,1385.223198,1285.993317,1015.497153,756.003124,978.022749,1223.004480,1274.641207,...,-0.002355,-0.002088,-0.011578,-0.031207,0.012907,-0.001701,-0.001606,-0.009225,0.000682,19607.725300
4,2023-11-22,39120.771619,1514.369207,1390.091520,1313.473886,1012.074146,757.428207,997.121127,1225.466864,1277.640903,...,0.002013,0.002353,0.012813,0.002121,0.007417,0.011286,0.010760,-0.024587,0.002998,19705.761619
5,2023-11-24,39157.260898,1521.864267,1380.354875,1316.106818,1023.381699,762.223976,997.325015,1230.306512,1279.390726,...,0.003949,0.001370,-0.001112,0.002973,0.008684,0.012772,0.004365,-0.019296,0.001495,19742.250898
6,2023-11-27,39122.081405,1537.100085,1379.046906,1307.714504,1034.958627,758.434870,1004.053528,1225.721576,1276.474440,...,-0.003727,-0.002279,0.003126,0.003768,0.005802,0.000072,-0.004790,0.009754,-0.034866,19707.071405
7,2023-11-28,39087.678008,1529.113535,1383.479276,1319.562447,1068.573880,752.428924,999.295967,1235.485862,1279.390726,...,0.007966,0.002285,0.010803,0.001151,-0.017678,0.007740,-0.002941,-0.008727,-0.042311,19672.668008
8,2023-11-29,39094.813592,1481.563016,1375.995127,1332.068497,1111.189211,767.913191,994.470479,1221.900774,1285.890325,...,-0.010996,0.005080,-0.010060,-0.000650,0.015344,0.004905,0.000894,0.006671,-0.006165,19679.803592
9,2023-11-30,39141.857289,1487.215012,1380.209521,1340.789990,1134.958347,775.615733,992.907199,1219.268669,1300.555680,...,-0.002154,0.011405,0.000158,-0.005452,-0.019403,0.014238,0.000938,-0.028459,0.011077,19726.847289


In [40]:
# Calculate the percentage gain/loss relative to the first row
results['Percentage_Gain_Loss'] = ((results['Total_Contribution'] - results['Total_Contribution'].iloc[0]) / results['Total_Contribution'].iloc[0])

results 

,Date,Cumulative_Portfolio_Value,CEG_Contribution,AAPL_Contribution,RTX_Contribution,PANW_Contribution,NOW_Contribution,AMZN_Contribution,CVX_Contribution,JPM_Contribution,...,JPM_Return,MSFT_Return,LOW_Return,FTNT_Return,ULTA_Return,CMI_Return,NVDA_Return,GEHC_Return,Total_Contribution,Percentage_Gain_Loss
0,2023-11-16,19415.01,1494.955770,1378.465710,1320.220680,931.920480,737.770380,970.750500,1203.730620,1261.975650,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19434.425010,0.000000
1,2023-11-17,38959.044496,1495.201562,1378.320355,1311.170133,952.266551,740.122943,986.722318,1226.570700,1273.391333,...,0.009046,-0.016801,0.007020,0.016660,0.009173,0.009310,-0.003678,0.015017,19544.034496,0.005640
2,2023-11-20,39109.491955,1502.450830,1391.108780,1302.777819,1002.189531,754.329103,993.179118,1225.891427,1277.307646,...,0.003075,0.020522,0.003633,0.012680,-0.008943,-0.005942,0.022536,0.004521,19694.481955,0.013381
3,2023-11-21,39022.7353,1505.276875,1385.223198,1285.993317,1015.497153,756.003124,978.022749,1223.004480,1274.641207,...,-0.002088,-0.011578,-0.031207,0.012907,-0.001701,-0.001606,-0.009225,0.000682,19607.725300,0.008917
4,2023-11-22,39120.771619,1514.369207,1390.091520,1313.473886,1012.074146,757.428207,997.121127,1225.466864,1277.640903,...,0.002353,0.012813,0.002121,0.007417,0.011286,0.010760,-0.024587,0.002998,19705.761619,0.013962
5,2023-11-24,39157.260898,1521.864267,1380.354875,1316.106818,1023.381699,762.223976,997.325015,1230.306512,1279.390726,...,0.001370,-0.001112,0.002973,0.008684,0.012772,0.004365,-0.019296,0.001495,19742.250898,0.015839
6,2023-11-27,39122.081405,1537.100085,1379.046906,1307.714504,1034.958627,758.434870,1004.053528,1225.721576,1276.474440,...,-0.002279,0.003126,0.003768,0.005802,0.000072,-0.004790,0.009754,-0.034866,19707.071405,0.014029
7,2023-11-28,39087.678008,1529.113535,1383.479276,1319.562447,1068.573880,752.428924,999.295967,1235.485862,1279.390726,...,0.002285,0.010803,0.001151,-0.017678,0.007740,-0.002941,-0.008727,-0.042311,19672.668008,0.012259
8,2023-11-29,39094.813592,1481.563016,1375.995127,1332.068497,1111.189211,767.913191,994.470479,1221.900774,1285.890325,...,0.005080,-0.010060,-0.000650,0.015344,0.004905,0.000894,0.006671,-0.006165,19679.803592,0.012626
9,2023-11-30,39141.857289,1487.215012,1380.209521,1340.789990,1134.958347,775.615733,992.907199,1219.268669,1300.555680,...,0.011405,0.000158,-0.005452,-0.019403,0.014238,0.000938,-0.028459,0.011077,19726.847289,0.015047


In [41]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(4)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,results)